In [1]:
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import time
import pandas as pd

In [2]:
months = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

In [3]:
years = [year for year in range(2006, 2021)]

In [4]:
strainer = SoupStrainer('a', class_ = 'SiteMapArticleList-link')

In [5]:
article_links = []
length = []
t0 = time()
for year in years:
    for month in months:
        print(f'{month} - {year}')
        t1 = time()
        for day in range(1,32):
            link = f'https://www.cnbc.com/site-map/{year}/{month}/{day}/'
            page = str(requests.get(link).content)
            page = page[str(page).find('body'):]
            soup = BeautifulSoup(page, parse_only = strainer)
            links = soup.find_all('a', class_='SiteMapArticleList-link')
            if links != []:
                for a_link in links:
                    article_links.append(a_link['href'])
                    length.append(link)
        print(time()-t1)
print(time()-t0)

January - 2006
19.736795902252197
February - 2006
15.59265685081482
March - 2006
15.232767105102539
April - 2006
14.59723424911499
May - 2006
13.847558975219727
June - 2006
14.088925838470459
July - 2006
14.160164833068848
August - 2006
14.10075831413269
September - 2006
14.962288856506348
October - 2006
20.142704248428345
November - 2006
14.173299312591553
December - 2006
14.994412183761597
January - 2007
16.32174277305603
February - 2007
17.09568452835083
March - 2007
16.70344305038452
April - 2007
17.841018199920654
May - 2007
20.132397174835205
June - 2007
17.873043298721313
July - 2007
14.924350500106812
August - 2007
14.971537828445435
September - 2007
14.558979272842407
October - 2007
13.781375408172607
November - 2007
13.241758108139038
December - 2007
14.031699895858765
January - 2008
14.995840549468994
February - 2008
13.943265438079834
March - 2008
14.127214193344116
April - 2008
21.185296058654785
May - 2008
14.62737774848938
June - 2008
15.509485244750977
July - 2008
22.92

In [6]:
#Writing all the links into a csv-file to avoid rescraping every time.
with open('article_links.csv', 'w') as a_file:
    for link in article_links:
        link = ''.join(link)
        a_file.write(link + '\n')

In [5]:
def scrape_content(link):
    #Load page
    page = requests.get(link).content
    page = page[str(page).find('body'):]
    soup_link = BeautifulSoup(page)
    
    #Get title
    #title = soup_link.find('h1').get_text()
    
    #Get date for publishing
    #date = soup_link.find('meta', itemprop="datePublished")['content'][:10]
    
    #Get article content
    #article = soup_link.find('div', itemprop='text').get_text()
    
    #return [title, date, article]
    return soup_link

In [2]:
article_links = pd.read_csv('article_links.csv', sep="<", header=None)
article_links.columns = ['links']
article_links = article_links['links'].to_list()
links = []
for link in article_links:
    if ".com/advertorial" not in link and ".com/select" not in link:
        links.append(link[0:len(link)-1])
article_links = links

In [ ]:
#t0 = time()
#for link in article_links[0:10]:
#    scrape_content(link)
#    print(time()-t0)

In [3]:
strainer = SoupStrainer('div', class_='Article')

In [15]:
df = []
index = 0
save_every = 1000
t0 = time()
for link in article_links[290000:295000]:
    t1 = time()
    request = requests.Session()
    page = request.get(link)
    status_code = page.status_code
    page = str(page.content)
    index += 1
    if status_code == 200:
        try:
            page = page[page.find('<div id="MainContent"'):]
            soup_link = BeautifulSoup(page)
            title = soup_link.find('h1', class_='ArticleHeader-headline').get_text()
            article = soup_link.find('div', class_="ArticleBody-articleBody").get_text()
            date = f'{link[29:31]}-{link[26:28]}-{link[21:25]}'
            topic = soup_link.find('a', class_="ArticleHeader-eyebrow").get_text()
            df.append([title, topic, date, article, link])
            t9 = time()
            print(f'({index}/5000) : {link}')
        except:
            pass
    else:
        print(f'({index}/5000) : Skipped')
    if index % save_every == 0:
        save = pd.DataFrame(df)
        file_name = f'articles_0_{index}.csv'
        save.to_csv(file_name)
print(time()-t0)

(1/5000) : https://www.cnbc.com/2018/03/26/oil-markets-middle-east-tension-global-market-recovery-in-focus.html
(2/5000) : https://www.cnbc.com/2018/03/27/why-one-of-wall-streets-biggest-bulls-says-hes-not-bullish-enough.html
(3/5000) : https://www.cnbc.com/2018/03/27/krawchecks-ellevate-rolls-out-funds-to-invest-in-women.html
(4/5000) : https://www.cnbc.com/2018/03/27/apple-education-event-in-chicago-tim-cooks-speech.html
(5/5000) : https://www.cnbc.com/2018/03/27/pharma-bro-martin-shkreli-files-notice-of-appeal-of-fraud-conviction.html
(6/5000) : https://www.cnbc.com/2018/03/27/cambridge-analytica-an-example-of-modern-day-colonialism-whistleblower.html
(7/5000) : https://www.cnbc.com/2018/03/27/linda-brown-central-figure-in-school-segregation-case-dies.html
(8/5000) : https://www.cnbc.com/2018/03/27/oscar-health-raises-165-million-at-3-point-2-billion-alphabet-founders.html
(10/5000) : https://www.cnbc.com/2018/03/27/facebook-ceo-mark-zuckerberg-will-reportedly-testify-before-congres

In [11]:
df = pd.DataFrame(df)
df

,0,1,2,3,4
0,Oil prices rise as geopolitical tensions offse...,Energy,26-03-2018,"Getty ImagesOil prices rose on Tuesday, holdin...",https://www.cnbc.com/2018/03/26/oil-markets-mi...
1,Why one of Wall Street\xe2\x80\x99s biggest bu...,Trading Nation,27-03-2018,VIDEO1:4001:40Strategist Tony Dwyer on his bul...,https://www.cnbc.com/2018/03/27/why-one-of-wal...
2,Krawcheck's Ellevest rolls out funds to invest...,Wall Street,27-03-2018,"Sally Krawcheck, CEO and co-founder of Elleves...",https://www.cnbc.com/2018/03/27/krawchecks-ell...
3,Apple reveals new $299 iPad at education-theme...,Tech,27-03-2018,VIDEO1:0201:02Apple reveals new $299 iPad at e...,https://www.cnbc.com/2018/03/27/apple-educatio...
4,Pharma bro Martin Shkreli appeals fraud convic...,Biotech and Pharma,27-03-2018,Ex-pharmaceutical executive Martin Shkreli arr...,https://www.cnbc.com/2018/03/27/pharma-bro-mar...
...,...,...,...,...,...
4264,Malaysia elections: Here are the results forec...,Elections,08-05-2018,Former Malaysian Prime Minister Najib RazakGet...,https://www.cnbc.com/2018/05/08/malaysian-elec...
4265,Trump vowed to punish companies that moved job...,Politics,08-05-2018,Republican presidential candidate Donald Trump...,https://www.cnbc.com/2018/05/08/trump-vowed-to...
4266,Treasury yields hold higher after Trump says h...,Bonds,08-05-2018,U.S. government debt yields ticked higher Tues...,https://www.cnbc.com/2018/05/08/iran-deal-anno...
4267,Trump told French President Macron he plans to...,Investing,08-05-2018,VIDEO2:1302:13Trump to pull out of Iran nuclea...,https://www.cnbc.com/2018/05/08/trump-told-fre...


In [12]:
df.to_csv('articles_290000_295000.csv')

In [13]:
df_test = pd.read_csv('articles_290000_295000.csv')

In [14]:
df_test.head(5)

,Unnamed: 0,0,1,2,3,4
0,0,Oil prices rise as geopolitical tensions offse...,Energy,26-03-2018,"Getty ImagesOil prices rose on Tuesday, holdin...",https://www.cnbc.com/2018/03/26/oil-markets-mi...
1,1,Why one of Wall Street\xe2\x80\x99s biggest bu...,Trading Nation,27-03-2018,VIDEO1:4001:40Strategist Tony Dwyer on his bul...,https://www.cnbc.com/2018/03/27/why-one-of-wal...
2,2,Krawcheck's Ellevest rolls out funds to invest...,Wall Street,27-03-2018,"Sally Krawcheck, CEO and co-founder of Elleves...",https://www.cnbc.com/2018/03/27/krawchecks-ell...
3,3,Apple reveals new $299 iPad at education-theme...,Tech,27-03-2018,VIDEO1:0201:02Apple reveals new $299 iPad at e...,https://www.cnbc.com/2018/03/27/apple-educatio...
4,4,Pharma bro Martin Shkreli appeals fraud convic...,Biotech and Pharma,27-03-2018,Ex-pharmaceutical executive Martin Shkreli arr...,https://www.cnbc.com/2018/03/27/pharma-bro-mar...
